## 데이터 준비

### 라이브러리 설치

In [ ]:
!pip install autokeras
# !pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 라이브러리/ 패키지 불러오기

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import glob

from math import sqrt
from PIL import Image

import cv2
import autokeras as ak

import tensorflow as tf
from tensorflow.keras.utils import plot_model

from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

### 데이터 불러오기

In [ ]:
class AutoImageClassification:
  def __init__(self, path):
    '''
      - 학습, 평가 데이터 준비
      - 학습 데이터의 경우 '학습'폴더 기반, 테스트 데이터의 경우 '테스트'폴더 기반
      - 정상/불량 폴더 내에 있는 리스트를 각각 1, 0에 해당하도록 target value를 설정하여 df에 저장
    '''
    self.train_df = self._create_df(path, '/학습')
    self.test_df = self._create_df(path, '/테스트')

  @staticmethod
  def _create_df(path, path2):
    '''
      이미지 데이터 불러오기를 진행하며 각각의 라벨링 데이터를 구성한다
    '''
    files1 = glob.glob(path + path2 + "/정상/*.png")
    files2 = glob.glob(path + path2 + "/불량/*.png")
    print('정상 : ', len(files1))
    print('불량 : ', len(files2))
    df_n = pd.DataFrame()
    df_p = pd.DataFrame()
    df_n['name'] = [x for x in files2]
    df_n['outcome'] = 0
    df_p['name'] = [x for x in files1]
    df_p['outcome'] = 1
    df = pd.concat([df_n, df_p], axis= 0, ignore_index= True)
    df = shuffle(df)
    return df

  def create_x_and_y(self):
    '''
      - 데이터 정제(전처리)
      - 이미지 데이터 → 학습 데이터로 변환
      - 이미지 데이터와 정상/불량 데이터를 합치는 code
    '''
    X = np.array([img_preprocess(p) for p in self.train_df.name.values])
    y = self.train_df.outcome.values
    return X, y

dir_path = os.path.join(os.getcwd(), 'resized')
aic = AutoImageClassification(path = dir_path)
# Numerical 데이터 파일 리스트 출력 화면

정상 :  1102
불량 :  59
정상 :  276
불량 :  15


In [ ]:
# # 드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# 파일 압축 풀기
# %cd 압축을 풀 경로
# !unzip -qq "압축파일 Path"

%cd /content/drive/MyDrive/6dogma/res
!unzip -qq "/content/drive/MyDrive/6dogma/res/resized.zip"

/content/drive/MyDrive/6dogma/res


### 이미지 데이터 불러오기 및 이미지 데이터 전처리

In [ ]:
def img_preprocess(img):
  # PIL 라이브러리를 통해 이미지 불러옴
  img = Image.open(img)
  # 이미지를 rgb 형태로 변환
  img = img.convert('RGB')
  # 이미지 사이즈 조정
  img = img.resize((256, 256))
  # 이미지 normalize 진행
  img = np.asarray(img)/255
  return img

## 알고리즘 선택

- autoKeras 내 ImageClassifier 딥러닝 모델
  - autokeras는 요즘 각광받고 있는 AutoML이 적용된 keras 라이브러리이다.
  - AutoML이란 Hyper-parameter를 스스로 변형하여 최적화 시키는 머신러닝을 의미한다.
    - 기존의 ML/DL의 경우 사용자가 변형시키면서 학습을 재진행하며 최적값을 찾아야 했다.
  - 하지만, 아직 최적화된 알고리즘이 아니기 때문에 기존의 ML/DL보다 정확도가 낮다는 단점이 있다.

## 모델링

### CNN 모델 모델링 & 모델 학습

In [ ]:
# is_training = True
# if is_training:
#   '''
#     - is_training이라는 trigger를 통해 학습을 진행할 것인지, 학습된 파일을 불러올 것인지 설정
#     - 학습 진행 시, 학습용 데이터 가져오도록 create_x_and_y()함수 실행
#     - max_trials: 해당 모델을 최대 몇 번 만들어서 진행할 것인지 결정
#     - loss: 어떤 방법으로 이미지 양/불 판정이므로 이진 분류로 진행
#     - overwrite: 기존에 진행한 모델이 있는 경우 덮어씌워서 학습할지 결정
#   '''
#   X, y = aic.create_x_and_y()
#   model = ak.ImageClassifier(max_trials=2, loss= 'binary_crossentropy',
#                              metrics='accuracy', overwrite= True)
#   model.fit(x=X, y=y, epochs=1, validation_split=0.2, verbose=2)
#   model = model.export_model()

#   try:
#     model.save("../res/image_classifier/model_autokeras", save_format='tf')
#   except Exception:
#     model.save("../res/image_classifier/model_autokeras.h5")


# else:
#   model = tf.keras.models.load_model("../res/image_classifier/model_keras", custom_objects=ak.CUSTOM_OBJECTS)

Trial 2 Complete [00h 00m 40s]
val_loss: 31.64508056640625

Best val_loss So Far: 0.11817537993192673
Total elapsed time: 00h 01m 04s


In [ ]:
X, y = aic.create_x_and_y()
clf = ak.ImageClassifier(max_trials=2, loss= 'binary_crossentropy',
                            metrics='accuracy', overwrite= True)
clf.fit(x=X, y=y, epochs=1, validation_split=0.2, verbose=2)
model = clf.export_model()

print(type(model))

try:
  model.save("../res/image_classifier/model_autokeras", save_format='tf')
except Exception:
  model.save("../res/image_classifier/model_autokeras.h5")

loaded_model = tf.keras.models.load_model("../res/image_classifier/model_keras", custom_objects=ak.CUSTOM_OBJECTS)

Trial 2 Complete [00h 00m 39s]
val_loss: 59.839595794677734

Best val_loss So Far: 0.1389896720647812
Total elapsed time: 00h 01m 06s


In [ ]:
print(type(model))

NameError: ignored

### CNN 모델 시각화

In [ ]:
model.summary()
# 어떠한 layer가 생성되었는지 확인 가능
# 각 layer의 output shape 확인 가능
# 학습되거나 학습되지 않는 parameter 개수 확인 가능

In [ ]:
plot_model(model)

## 모델 튜닝

### CNN 모델 튜닝

- CNN 모델의 경우 튜닝할 것이 max_trials로 정해져 있으며, 이는 최대 시도 회수로 epoch와는 다른 개념으로 사용된다.
- max_trials는 모델 구조에 따라 달라질 수 있으며, 최대 횟수 중 가장 좋은 모델을 최종적으로 선택하기 때문에 많이 한다고 해서 좋은 것이 아니다.

## 모델 평가 및 해석

### CNN 모델 평가

In [ ]:
x_test = np.array